<a href="https://colab.research.google.com/github/rahiakela/genai-research-and-practice/blob/main/building-natural-language-and-llm-pipelines/02-diving-deep-into-llm/jupyter-notebooks/02_create-simple-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys.

# Your First AI Agent: Getting Started

## Welcome! 👋

In this notebook, you'll create your very first AI agent - a program that can think, use tools, and answer questions.

## What is an Agent?

An **agent** is an AI program that can:
- Understand what you ask
- Decide what tools it needs
- Use those tools to find answers
- Give you a helpful response

Think of it like a smart assistant that knows when to search Google, when to do math, and when to just answer from what it knows.

## What You'll Build

By the end of this notebook, you'll have an agent that can:
1. Search the web for current information
2. Answer questions using its knowledge
3. Run completely locally on your machine with Ollama

## Prerequisites

Make sure you have:
- **Ollama installed**: Download from [ollama.com](https://ollama.com)
- **Mistral-Nemo model**: Run `ollama pull mistral-nemo:12b` in your terminal. Select an LLM, SLM or RLM from this list [https://ollama.com/search](https://ollama.com/search)
- **Tavily API key**: Get a free key from [tavily.com](https://tavily.com) for web search

Let's get started!

## Step 0: Setup

In [ ]:
!pip install langchain-ollama
!pip install langchain-community

In [ ]:
!sudo apt-get install zstd

In [2]:
!sudo apt-get update && sudo apt-get install -y zstd cuda-drivers
!curl -fsSL https://ollama.ai/install.sh | sh
import os
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,751 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [39.2 kB]
Get:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main 

In [1]:
!nohup /usr/local/bin/ollama serve > ollama.log 2>&1 &

In [2]:
!ollama pull mistral-nemo:12b

In [3]:
!ollama pull nomic-embed-text

## Step 1: Import What We Need



Let's import the pieces we need to build our agent.

In [4]:
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults

from google.colab import userdata

print("✓ Ready to build an agent!")

✓ Ready to build an agent!


## Step 2: Give Your Agent a Tool

Let's give our agent a **search tool** so it can look up current information on the web.

In [7]:
# Create a search tool that can look things up on the web
search_tool = TavilySearchResults(max_results=3, tavily_api_key=userdata.get('TAVILY_API_KEY'))

# Put it in a list (we can add more tools later)
tools = [search_tool]

print(f"✓ Your agent has {len(tools)} tool available: search!")

✓ Your agent has 1 tool available: search!


## Step 3: Create Your Agent with Ollama

Now for the magic! Let's create an agent that runs locally using Ollama.

In [8]:
# Create a local model with Ollama
model = ChatOllama(
    model="mistral-nemo:12b",
    temperature=0
)

# Create an agent - it's this simple!
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="You are a helpful assistant. Use the search tool when you need current information."
)

print("✓ Your agent is ready!")
print("  Running locally with Ollama!")
print("  It can think, search, and answer questions.")

✓ Your agent is ready!
  Running locally with Ollama!
  It can think, search, and answer questions.


## Step 4: Ask Your Agent a Question

Let's try it out! We'll ask a question that needs current information.

In [9]:
# Ask a question
query = "What are the latest developments in AI in 2026?"

print(f"Question: {query}\n")
print("⚙️  Thinking and searching...\n")

# Get the answer
result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

# Show the response
print("Answer:")
print(result["messages"][-1].content)

Question: What are the latest developments in AI in 2026?

⚙️  Thinking and searching...

Answer:
Here are some of the latest developments in AI as predicted for 2026:

1. **Open Source Competition**: Chinese AI firms like Zhipu and Moonshot are following DeepSeek's lead to open-source their models, pushing American firms to do the same. Alibaba's Qwen family has become an open-source powerhouse with its wide range of model sizes and specialized versions.

2. **AI-driven Innovation**:
   - Google DeepMind used AlphaEvolve to improve data center management and TPU chip efficiency.
   - Open-source versions of AlphaEvolve have been released, including OpenEvolve by Asankhaya Sharma and SinkaEvolve by Sakana AI.
   - A team of US and Chinese researchers revealed AlphaResearch, claiming it improves on one of AlphaEvolve's math solutions.

3. **Hybrid Computing**: The rise of hybrid computing, where quantum works alongside AI and supercomputers, is expected to drive greater accuracy in mode

## Step 5: Ask Without Needing Search

The agent is smart enough to know when it doesn't need to search!

In [10]:
# A simple math question
simple_query = "What is 25 times 4?"

result = agent.invoke({
    "messages": [{"role": "user", "content": simple_query}]
})

print(f"Question: {simple_query}")
print(f"Answer: {result['messages'][-1].content}")
print("\n✓ The agent answered directly without searching!")

Question: What is 25 times 4?
Answer: The result of 25 multiplied by 4 is 100.

✓ The agent answered directly without searching!


## 🎉 Congratulations!

You just built your first AI agent! Here's what you learned:

### What You Did
✅ Created an AI agent with `create_agent()`  
✅ Gave it a tool to search the web  
✅ Asked questions and got intelligent answers  
✅ Saw how the agent decides when to use tools  
✅ Had a conversation with memory

### What's Happening Behind the Scenes?
When you ask a question, the agent:
1. **Thinks**: "Do I know this, or do I need to search?"
2. **Decides**: Uses tools if needed, or answers directly
3. **Responds**: Gives you a helpful answer
4. **Remembers**: Keeps track of the conversation

### Next Steps
In the next notebooks, you'll learn:
- How agents work under the hood (state graphs)
- How to give agents more tools
- How to create multi-agent systems

### Try It Yourself!
Experiment with different questions:
- "What's the weather in Tokyo today?" (needs search)
- "What is 15% of 200?" (can answer directly)
- "Tell me about AI" then "What are some examples?" (uses memory)

In [11]:
query = "What's the weather in New Delhi today?"

print(f"Question: {query}\n")
print("⚙️  Thinking and searching...\n")

# Get the answer
result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

# Show the response
print("Answer:")
print(result["messages"][-1].content)

Question: What's the weather in New Delhi today?

⚙️  Thinking and searching...

Answer:
Here's the weather forecast for New Delhi today:

- **Temperature**: High of 25°C and low of 13°C
- **Weather**: Partly cloudy with a chance of patchy light rain and thunder


In [12]:
simple_query = "What is 15% of 200?"

result = agent.invoke({
    "messages": [{"role": "user", "content": simple_query}]
})

print(f"Question: {simple_query}")
print(f"Answer: {result['messages'][-1].content}")
print("\n✓ The agent answered directly without searching!")

Question: What is 15% of 200?
Answer: To find 15% of 200, you can follow these steps:

1. Convert the percentage to a decimal by dividing by 100:
   \[
   15\% = \frac{15}{100} = 0.15
   \]

2. Multiply the decimal by the number:
   \[
   x = 200 \times 0.15
   \]

3. Calculate the product:
   \[
   x = 30
   \]

So, 15% of 200 is 30.

Alternatively, you can use the relationship between percent, part, and whole to solve for the unknown number:

1. Divide the percentage by 100:
   \[
   \frac{15}{100} = 0.15
   \]

2. Multiply the result by the number you want to find the percentage of:
   \[
   x = 200 \times 0.15
   \]

3. Calculate the product:
   \[
   x = 30
   \]

Hence, 15% of 200 is 30.

Another method is to use keywords and multiplication:

1. Identify the keyword "of" which suggests multiplication.
2. Convert the percentage to a decimal:
   \[
   15\% = 0.15
   \]

3. Multiply the number by the decimal:
   \[
   x = 200 \times 0.15
   \]

4. Calculate the product:
   \[
   x =

In [13]:
query = "Tell me about AI?"

print(f"Question: {query}\n")
print("⚙️  Thinking and searching...\n")

# Get the answer
result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

# Show the response
print("Answer:")
print(result["messages"][-1].content)

Question: Tell me about AI?

⚙️  Thinking and searching...

Answer:
Here's a summary of the key points about Artificial Intelligence (AI) from the provided sources:

**Definition:**
- AI is focused on creating smart machines that can perform tasks typically requiring human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making.
- Instead of relying on explicit instructions, AI systems learn patterns from vast amounts of data to make predictions or decisions.

**Types of AI:**
1. **Narrow or Weak AI:** Designed to perform a single task (e.g., facial recognition, internet searches).
2. **General or Strong AI:** Understands and learns any intellectual task that a human can do.
3. **Superintelligent AI:** Significantly surpasses human intelligence in reasoning, creativity, and emotional intelligence.

**Applications of AI:**
- Everyday use: Virtual assistants, personalized recommendations, spam filters, navigation apps.
- Healthcare: Disease diagnosis, 

In [14]:
query = "What are some examples?"

print(f"Question: {query}\n")
print("⚙️  Thinking and searching...\n")

# Get the answer
result = agent.invoke({
    "messages": [{"role": "user", "content": query}]
})

# Show the response
print("Answer:")
print(result["messages"][-1].content)

Question: What are some examples?

⚙️  Thinking and searching...

Answer:
Here are some examples of current events from various sources:

1. **The New York Times:**
   - **Title:** Current Events - The New York Times
   - **Content:**
     - Students share their feelings about America following two fatal shootings involving federal agents.
     - A series of trials will explore if social media companies should be held responsible for teens' mental health struggles.
     - Harvard proposes curbing grade inflation by limiting the number of A grades professors can award. What do you think?
   - **Link:** https://www.nytimes.com/spotlight/learning-current-events

2. **CNN:**
   - **Title:** Breaking News, Latest News and Videos
   - **Content:**
     - A total solar eclipse swept across North America on April 8, 2024.
     - SpaceX's Crew-12 Dragon successfully docked at the International Space Station on February 14.
     - Former US Secretary of State Hillary Clinton spoke to the BBC at 